In [1]:
import os
import pyvtk
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from netCDF4 import Dataset
import shutil
import glob

In [2]:
# The data structure in element-wise output is too complicated for xarray.open_mfdataset.
# Here we open the files as individual datasets and concatenate them on the variable level.
# This code is compatible with parallel netcdf build (single file output)

# load_wave_data=True:  read wave data and return numpy.ndarray
# load_wave_data=False: do not read wave data and return xarray.DataArray (use False if data is big)

def read_element_output(data_dir, load_wave_data=True):
    ################ open files ################
    # filenames
    nc_fnames = [f for f in os.listdir(data_dir) if 'axisem3d_synthetics.nc' in f]
    # print('files to open: ', nc_fnames)

    # open files
    nc_files = []
    for nc_fname in nc_fnames:
        nc_files.append(xr.open_dataset(data_dir + '/' + nc_fname))
    
    ################ variables that are the same in the datasets ################
    # read Na grid (all azimuthal dimensions)
    na_grid = nc_files[0].data_vars['list_na_grid'].values.astype(int)

    # read time
    data_time = nc_files[0].data_vars['data_time'].values
    
    
    ################ variables to be concatenated over the datasets ################
    # define empty lists of xarray.DataArray objects
    xda_list_element_na = []
    xda_list_element_coords = []
    dict_xda_list_element = {}
    dict_xda_data_wave = {}
    for nag in na_grid:
        dict_xda_list_element[nag] = []
        dict_xda_data_wave[nag] = []
    
    # loop over nc files
    for nc_file in nc_files:
        # append DataArrays
        xda_list_element_na.append(nc_file.data_vars['list_element_na'])
        xda_list_element_coords.append(nc_file.data_vars['list_element_coords'])
        for nag in na_grid:
            dict_xda_list_element[nag].append(nc_file.data_vars['list_element__NaG=%d' % nag])
            dict_xda_data_wave[nag].append(nc_file.data_vars['data_wave__NaG=%d' % nag])
            
    # concat xarray.DataArray
    xda_list_element_na = xr.concat(xda_list_element_na, dim='dim_element')
    xda_list_element_coords = xr.concat(xda_list_element_coords, dim='dim_element')
    for nag in na_grid:
        dict_xda_list_element[nag] = xr.concat(dict_xda_list_element[nag], dim='dim_element__NaG=%d' % nag)
        dict_xda_data_wave[nag] = xr.concat(dict_xda_data_wave[nag], dim='dim_element__NaG=%d' % nag)
        
    # read data to numpy.ndarray
    list_element_na = xda_list_element_na.values.astype(int)
    list_element_coords = xda_list_element_coords.values
    dict_list_element = {}
    dict_data_wave = {}
    for nag in na_grid:
        dict_list_element[nag] = dict_xda_list_element[nag].values.astype(int)
        if load_wave_data:
            dict_data_wave[nag] = dict_xda_data_wave[nag].values
        
    ############### return ################
    if load_wave_data:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_data_wave
    else:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_xda_data_wave

# Inplane slices

## Read

In [3]:
# data dir
# ModelNameList = ['model0021']
ModelNameList = []
for imodel in [3186, 3624, 4082, 5817, 6107, 9223, 8609, 8610, 8611, 8612, 8613, 8614, 8615, 8616, 8617, 3309, 4363, 5894, 6419, 9720]:
    ModeName = 'LatinSphericalHarmonicsAcousticBall%04d' %imodel
    ModelNameList.append(ModeName)
    
# wave dimension to animation
output_channel = 'X'
wave_dim_X = output_channel.index('X')

# output_channel = 'Xspz'
# wave_dim_s = output_channel.index('s')
# wave_dim_p = output_channel.index('p')
# wave_dim_z = output_channel.index('z')
# wave_dim_X = output_channel.index('X')

for ModelName in ModelNameList:
    data_dir = '../Runs/%s/output/elements/full_coeff' %ModelName
    
    if not os.path.exists(data_dir):
        print('ERROR %s not found' %data_dir)
        continue
    
    # read
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(data_dir)

        # print(ModelName, "reading error!!!!!")
        # continue

    # time steps
    ntime = len(data_time)

    # # phi of the slices
    nag = na_grid[0]

    # GLL coords on elements
    nelem = list_element_coords.shape[0]
    ngll = list_element_coords.shape[1]
    # flattened coords, (s, z)
    element_coords_sz = list_element_coords.reshape((nelem * ngll), 2)

    # # save element_coords_sz
    # element_coords_sz

    wave_X = np.ndarray((nag, nelem * ngll))

    for itime in np.arange(ntime):

        for ielem in np.arange(nelem):
            wave_X[:, (ielem * ngll):(ielem * ngll + ngll)] = dict_data_wave[nag][ielem, :, :, wave_dim_X, itime]


        # make slice for phi
        NETCDFDir = data_dir + '/netcdf_coeff'
        os.makedirs(NETCDFDir, exist_ok=True)

        nc = Dataset(NETCDFDir+'/phi_coef_time%d.nc' %itime, 'w')
        nc.createDimension('npoint', size=nelem * ngll)
        nc.createDimension('Nr_Dim', size=nag)
        nc.createDimension('d2', size=2)
        # nc.createDimension('ntime', size=ntime)

        nc.createVariable('element_coords_sz', float, dimensions=('npoint','d2'))
        nc['element_coords_sz'][:] = element_coords_sz[:]
        # nc.createVariable('time', float, dimensions=('ntime'))
        # nc['time'][:] = data_time[:]

        nc.createVariable('X_coef', float, dimensions=('Nr_Dim', 'npoint'))
        nc['X_coef'][:] = wave_X[:]

        nc.close()

    # # Check Repeated Dataset
    # if os.path.exists('../DataSet/%s/' %ModelName):
    #     shutil.rmtree('../DataSet/%s/' %ModelName)

    # Initial Model Folder
    os.makedirs('../DataSet/%s/' %ModelName, exist_ok=True)

    # Make time info file
    if os.path.exists('../DataSet/%s/time_info.nc' %ModelName):
        os.remove('../DataSet/%s/time_info.nc' %ModelName)
    nc = Dataset('../DataSet/%s/time_info.nc' %ModelName, 'w')
    nc.createDimension('ntime', size=ntime)
    nc.createVariable('time', float, dimensions=('ntime'))
    nc['time'][:] = data_time[:]

    # Check 3D model nc file
    NCFilePath = glob.glob('../Runs/%s/input/*.nc' %ModelName)
    if len(NCFilePath) > 0:
        for path in NCFilePath:
            shutil.copy(path, '../DataSet/%s/' %ModelName)
    # Move Spherical Harmonics Paramters
    SphericalHarmonicsPath = glob.glob('../Runs/%s/Spherical_Harmonics.pkl' %ModelName)[0]
    shutil.copy(SphericalHarmonicsPath, '../DataSet/%s/' %ModelName)

    target_dir = '../DataSet/%s/snapshot_coeff/' %ModelName
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    shutil.move(data_dir + '/netcdf_coeff/', target_dir)

    print(data_dir + '/netcdf_coeff/', target_dir)

../Runs/LatinSphericalHarmonicsAcousticBall3186/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall3186/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall3624/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall3624/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall4082/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall4082/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall5817/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall5817/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall6107/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall6107/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall9223/output/elements/full_coeff/netcdf_coeff/ ../DataSet/LatinSphericalHarmonicsAcousticBall9223/snapshot_coeff/
../Runs/LatinSphericalHarmonicsAcousticBall8609/output/elements/

In [9]:
data_time

array([0.0748784, 0.2747896, 0.4747008, 0.674612 , 0.8745232, 1.0744344,
       1.2743456, 1.4742568, 1.674168 , 1.8740792, 2.0739904, 2.2739016,
       2.4738128, 2.673724 , 2.8736352])

In [27]:
wave_X[:,10]

array([ 3.17100799e-08, -5.53862955e-09, -9.56091117e-09,  4.23856894e-10,
        1.93133931e-09,  3.39408723e-10, -2.14357074e-10, -2.58024487e-11,
       -1.94617603e-11,  1.30911636e-11, -3.95640222e-11, -7.43270202e-12,
        2.94443636e-11,  1.28083603e-11, -7.14774333e-12, -1.02439940e-11])

In [20]:
        # make slice for phi
        NETCDFDir = data_dir + '/netcdf'
        os.makedirs(NETCDFDir, exist_ok=True)

        nc = Dataset(NETCDFDir+'/phi_coef_time%d.nc' %itime, 'w')
        nc.createDimension('npoint', size=nelem * ngll)
        nc.createDimension('Nr_Dim', size=nag)

        # nc.createVariable('x', float, dimensions=('npoint'))
        # nc['x'][:] = xyz[:,0]
        # nc.createVariable('y', float, dimensions=('npoint'))
        # nc['y'][:] = xyz[:,1]
        # nc.createVariable('z', float, dimensions=('npoint'))
        # nc['z'][:] = xyz[:,2]
        # nc.createVariable('time', float, dimensions=('npoint'))
        # nc['time'][:] = np.ones(len(xyz))*data_time[itime]

        nc.createVariable('X', float, dimensions=('npoint','Nr_Dim'))
        print(np.shape(nc['X'][:]))
        nc['X'][:] = wave_X

(3648, 16)


In [29]:
element_coords_sz[:,1].min(), element_coords_sz[:,1].max()

(-987.4046669297907, 987.4046669297908)

In [6]:
np.shape(wave_X)

(3648, 16, 15)

In [5]:
np.shape(element_coords_sz)

(3648, 2)

In [6]:
np.shape(data_time)

(15,)

In [7]:
np.shape(list_element_na)

(3648, 5)

In [8]:
np.shape(list_element_coords)

(3648, 1, 2)

In [9]:
np.shape(dict_list_element)

()

In [10]:
np.shape(dict_data_wave)

()

In [24]:
dict_data_wave[16][10, 0, :, 0, :]
wave_X[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nag][0, 0, :, wave_dim_X, :]
        

nag = 16

dict_data_wave[nag][10, 0, :, 0, :]

NameError: name 'ielem' is not defined

In [37]:
ielem = 10
itime = 10
np.shape(dict_data_wave[nag][ielem, :, :, wave_dim_X, itime])

(16, 1)